## Data pre-processing

In [79]:
# Import necessary libraries
import pandas as pd

In [80]:
df = pd.read_csv("../data/interim/nhanes_data_interim.csv")

### Convert categorical variables represented by float to int

In [81]:
df["diabetes"] = df["diabetes"].astype(int)
df["current_smoking_freq"] = df["current_smoking_freq"].astype(int)

### Identify outliers

In [82]:
def three_sd_range(series):
    mean = series.mean()
    sd = series.std()
    lower_bound = mean - 3 * sd
    upper_bound = mean + 3 * sd
    
    return (lower_bound, upper_bound)

In [83]:
for col_name in ["bmi", "age", "hba1c_percentage", "fpg_mmol_L", "hdl_mmol_L"]:
    lower_bound, upper_bound = three_sd_range(df[col_name])
    outlier_present = (df[col_name].min() < lower_bound or df[col_name].max() > upper_bound)
    print(f"{col_name} has at least one outlier: {outlier_present}")
    
    if outlier_present:
        curr_length = df[col_name].count()
        df = df[(df[col_name] >= lower_bound) & (df[col_name] <= upper_bound)]
        new_length = df[col_name].count()
        print(f"{curr_length - new_length} row has been removed")
    
    print()

bmi has at least one outlier: True
11 row has been removed

age has at least one outlier: False

hba1c_percentage has at least one outlier: True
32 row has been removed

fpg_mmol_L has at least one outlier: True
30 row has been removed

hdl_mmol_L has at least one outlier: True
16 row has been removed



In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1120 entries, 0 to 1208
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   bmi                       1120 non-null   float64
 1   high_blood_pressure       1120 non-null   int64  
 2   gender                    1120 non-null   int64  
 3   age                       1120 non-null   int64  
 4   diabetes                  1120 non-null   int32  
 5   hba1c_percentage          1120 non-null   float64
 6   fpg_mmol_L                1120 non-null   float64
 7   hdl_mmol_L                1120 non-null   float64
 8   lifetime_100_cigs_smoked  1120 non-null   int64  
 9   current_smoking_freq      1120 non-null   int32  
dtypes: float64(4), int32(2), int64(4)
memory usage: 87.5 KB


In [85]:
# Save the pre-processed DataFrame to a new CSV file
df.to_csv("../data/processed/nhanes_data_processed.csv", index=False)

### Categorical encoding

In [86]:
# Map multiclass values to more readable strings to prepare for one-hot encoding
df["diabetes"] = df["diabetes"].map({1: "yes", 2: "no", 3: "borderline"})
df["current_smoking_freq"] = df["current_smoking_freq"].map({1: "every_day", 2: "some_days", 3: "none"})

In [87]:
# One-hot encode the multiclass columns "diabetes" and "current_smoking_frequency"
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, dtype=int)
one_hot_encoded = encoder.fit_transform(df[["diabetes", "current_smoking_freq"]])

encoded_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(["diabetes", "current_smoking_freq"]))

# Concatenate the original DataFrame with the one-hot encoded DataFrame
df = pd.concat([df, encoded_df], axis=1)

# Drop the original multiclass columns after encoding
df = df.drop(columns=["diabetes", "current_smoking_freq"], axis=1)

In [88]:
# Resultant DataFrame after one-hot encoding
df.sample(5)

,bmi,high_blood_pressure,gender,age,hba1c_percentage,fpg_mmol_L,hdl_mmol_L,lifetime_100_cigs_smoked,diabetes_borderline,diabetes_no,diabetes_yes,current_smoking_freq_every_day,current_smoking_freq_none,current_smoking_freq_some_days
286,40.9,1.0,1.0,55.0,5.8,5.16,1.16,1.0,0.0,1.0,0.0,0.0,1.0,0.0
389,38.6,0.0,0.0,55.0,5.4,5.33,0.96,1.0,0.0,1.0,0.0,0.0,1.0,0.0
986,21.3,1.0,1.0,68.0,6.2,7.05,1.42,1.0,0.0,1.0,0.0,0.0,1.0,0.0
520,25.8,0.0,0.0,64.0,5.6,5.94,1.55,1.0,0.0,1.0,0.0,0.0,1.0,0.0
396,28.8,1.0,0.0,59.0,5.8,5.72,2.17,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [89]:
# Save the one-hot encoded pre-processed DataFrame to a new CSV file
df.to_csv("../data/processed/nhanes_data_processed_one_hot.csv", index=False)